## Set up the experiment folder

In [3]:
import os, shutil

# Create a folder for the experiment files
training_folder = 'driver-training'
os.makedirs(training_folder, exist_ok=True)

# Copy the data file into the experiment folder
shutil.copy('data/porto_seguro_safe_driver_prediction_train.csv', os.path.join(training_folder, "porto_seguro_safe_driver_prediction_train.csv"))


'driver-training/porto_seguro_safe_driver_prediction_train.csv'

## train.py
This file defines the key functions required to train the model.  
The file can be invoked with `python train.py` for development purposes.

In [4]:
%%writefile $training_folder/train.py
import os
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.model_selection import train_test_split
import lightgbm


def split_data(data_df):
    """Split a dataframe into training and validation datasets"""
    
    ## TODO
    features = data_df.drop(['target', 'id'], axis = 1)
    labels = np.array(data_df['target'])
    features_train, features_valid, labels_train, labels_valid = train_test_split(features, labels, test_size=0.2, random_state=0)

    train_data = lightgbm.Dataset(features_train, label=labels_train)
    valid_data = lightgbm.Dataset(features_valid, label=labels_valid, free_raw_data=False)
    return (train_data, valid_data)


def train_model(data, parameters):
    """Train a model with the given datasets and parameters"""
    # The object returned by split_data is a tuple.
    # Access train_data with data[0] and valid_data with data[1]
    
    ## TODO
    #train_data, valid_data = split_data(data)
    model = lightgbm.train(parameters,
                           data[0],
                           valid_sets=data[1],
                           num_boost_round=500,
                           early_stopping_rounds=20)
    return model


def get_model_metrics(model, data):
    """Construct a dictionary of metrics for the model"""
    
    ## TODO
    
    predictions = model.predict(data[1].data)
    fpr, tpr, thresholds = metrics.roc_curve(data[1].label, predictions)
    model_metrics = {"auc": (metrics.auc(fpr, tpr))}
    print(model_metrics)
    return model_metrics


def main():
    """This method invokes the training functions for development purposes"""
    
    # Read data from a file
    data_df = pd.read_csv('porto_seguro_safe_driver_prediction_train.csv')

    # Hard code the parameters for training the model
    parameters = {
        'learning_rate': 0.02,
        'boosting_type': 'gbdt',
        'objective': 'binary',
        'metric': 'auc',
        'sub_feature': 0.7,
        'num_leaves': 60,
        'min_data': 100,
        'min_hessian': 1,
        'verbose': 2
    }

    # Call the functions defined in this file
    ## TODO
    data = split_data(data_df)
    modeltrain = train_model(data, parameters)    
    # Print the resulting metrics for the model
    ## TODO
    get_model_metrics(model, data)
    
if __name__ == '__main__':
    main()


Overwriting driver-training/train.py


## parameters.json
This file will specify the parameters used to train the model.

In [4]:
%%writefile $training_folder/parameters.json
{
    "training":
    {
        "learning_rate": 0.04,
        "boosting_type": "gbdt",
        "objective": "binary",
        "metric": "auc",
        "sub_feature": 0.7,
        "num_leaves": 60,
        "min_data": 100,
        "min_hessian": 1,
        "verbose": 0
    }
}


Overwriting driver-training/parameters.json


## driver_training.py
This file will be the entry script when running an Azure ML context.  
It calls the functions defined in train.py for data preparation and training, but reads parameters from a file, and logs output to the Azure ML context.  
The file can be invoked with `python driver_training.py` for development purposes.

In [5]:
%%writefile $training_folder/driver_training.py
# Import libraries
import argparse
from azureml.core import Run
import joblib
import json
import os
import pandas as pd
import shutil

# Import functions from train.py
from train import split_data, train_model, get_model_metrics

# Get the output folder for the model from the '--output_folder' parameter
parser = argparse.ArgumentParser()
parser.add_argument('--output_folder', type=str, dest='output_folder', default="outputs")
args = parser.parse_args()
output_folder = args.output_folder

# Get the experiment run context
run = Run.get_context()

# load the safe driver prediction dataset
train_df = pd.read_csv('porto_seguro_safe_driver_prediction_train.csv')

# Load the parameters for training the model from the file
with open("parameters.json") as f:
    pars = json.load(f)
    parameters = pars["training"]

# Log each of the parameters to the run
for param_name, param_value in parameters.items():
    run.log(param_name, param_value)
    
# Use the functions imported from train.py to prepare data, train the model, and calculate the metrics
## TODO

    model = train_model(train_df, parameters)
    train_data, valid_data = split_data(train_df)
    
    # Print the resulting metrics for the model
    ## TODO
    get_model_metrics(model, valid_data)
    
# Save the trained model to the output folder
    os.makedirs(output_folder, exist_ok=True)
    output_path = output_folder + "/driver_model.pkl"
    joblib.dump(value=model, filename=output_path)

    run.complete()

Overwriting driver-training/driver_training.py


In [6]:
import azureml.core
from azureml.core import Workspace

# Load the workspace
ws = Workspace.from_config()

## Use an Estimator to Run the Script as an Experiment

See [this tutorial](https://github.com/MicrosoftDocs/mslearn-aml-labs/blob/master/02-Training_Models.ipynb) for a starting point

Use the scikit-learn and lightgbm conda packages

In [9]:
from azureml.train.estimator import Estimator
from azureml.core import Experiment

# Create an estimator
estimator = Estimator(source_directory=training_folder,
                      entry_script='driver_training.py',
                      compute_target='local',
                      conda_packages=['scikit-learn','lightgbm']
                      )

# Create an experiment
experiment_name = 'driver-training'
experiment = Experiment(workspace = ws, name = experiment_name)

# Run the experiment based on the estimator
run = experiment.submit(config=estimator)
run.wait_for_completion(show_output=True)



TrainingException: TrainingException:
	Message: {
    "error_details": "<html>\r\n<head><title>500 Internal Server Error</title></head>\r\n<body>\r\n<center><h1>500 Internal Server Error</h1></center>\r\n<hr><center>nginx</center>\r\n</body>\r\n</html>\r\n",
    "status_code": 500,
    "url": "https://westus2.experiments.azureml.net/content/v1.0/subscriptions/b4f30574-19b5-4753-926d-877888e82fc4/resourceGroups/oh-dsdata-data/providers/Microsoft.MachineLearningServices/workspaces/team5ws/snapshots/latest/metadata"
}
	InnerException ExperimentExecutionException:
	Message: {
    "error_details": "<html>\r\n<head><title>500 Internal Server Error</title></head>\r\n<body>\r\n<center><h1>500 Internal Server Error</h1></center>\r\n<hr><center>nginx</center>\r\n</body>\r\n</html>\r\n",
    "status_code": 500,
    "url": "https://westus2.experiments.azureml.net/content/v1.0/subscriptions/b4f30574-19b5-4753-926d-877888e82fc4/resourceGroups/oh-dsdata-data/providers/Microsoft.MachineLearningService

In [8]:
# Print the resulting metrics
metrics = run.get_metrics()
for k, v in metrics.items():
        print(k, v)

NameError: name 'run' is not defined

In [14]:
# Register the model
run.register_model(model_path='outputs/driver_model.pkl', model_name='driver_model.pkl')

Model(workspace=Workspace.create(name='team5ws', subscription_id='b4f30574-19b5-4753-926d-877888e82fc4', resource_group='oh-dsdata-data'), name=driver_model.pkl, id=driver_model.pkl:1, version=1, tags={}, properties={})

In [5]:
!pytest driver-training

============================= test session starts ==============================
platform linux -- Python 3.6.9, pytest-5.4.1, py-1.8.0, pluggy-0.13.0
rootdir: /mnt/batch/tasks/shared/LS_root/mounts/clusters/devclusterteam5/code
plugins: arraydiff-0.3, openfiles-0.4.0, doctestplus-0.4.0, remotedata-0.3.2
collected 3 items                                                              

driver-training/test_train.py ...                                        [100%]

=============================== warnings summary ===============================
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/nose/importer.py:12
  /anaconda/envs/azureml_py36/lib/python3.6/site-packages/nose/importer.py:12: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
    from imp import find_module, load_module, acquire_lock, release_lock

-- Docs: https://docs.pytest.org/en/latest/warnings.html
========================= 3 passed, 1 wa